In [1]:
import pandas as pd
import numpy as np
import datetime
import pandas_datareader as pdr
# import matplotlib.pyplot as plt
# import statsmodels.api as sm

import bs4 as bs
import requests

# from IPython.display import Math
# from IPython.display import Latex
# from IPython.display import IFrame
risk_free_rate = .0227

## Class Methods

In [4]:
class DataGenerator:
    def __init__(self, tickers, start_date, end_date, calculate=True):
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self._calculate = calculate
        
        if calculate:
            self.data = self.getStockData(tickers, start_date, end_date)
            self.data['Returns'] = (self.data['Adj Close'].pct_change() + 1)
            self.data = self.data.fillna(1)
            for t in self.tickers:
                self.data.loc[t].head(1)['Returns'] = 1.0
        else:
            self.data = pd.DataFrame()
        
    def getStockData(self, tickers, start_date, end_date):
        def data(ticker):
            try:
                return (pdr.get_data_yahoo(ticker, start=start_date, end=end_date))
            except (KeyError):
                return pd.DataFrame(index=pd.date_range(start=start_date,end=end_date),columns=["High", "Low", "Open", "Close", "Volume", "Adj Close"])
            
        datas = map (data, tickers)
        return pd.concat(datas, keys=tickers, names=['Ticker', 'Date'])
    
    def xs(self, idx):
        return self.data.xs(idx, level=1, drop_level=False)
    
    def get_tickers(self):
        return self.tickers
    
    def df(self):
        return self.data

    def __str__(self):
        return str(self.data)

NameError: name 'pd' is not defined

In [5]:
# # d = getStockData(sp500_tickers(), datetime.datetime(2019, 10, 1), datetime.datetime(2019, 10, 5))

# tickers = ['AAPL','BKR', 'MSFT']
# start_date = datetime.datetime(2019, 9, 1)
# end_date = datetime.datetime(2019, 9, 10)

# def data(ticker):
#     try:
#         return pdr.get_data_yahoo(ticker, start=start_date, end=end_date)
#     except (KeyError):

# datas = list(map (data, tickers))
# status = False
# redo = []

# # for i in range(len(datas)):
# #     if datas[i].empty:
# #         if not status:
# #             redo.append(i)
# #         else:
# #             print(i)
# #             datas[i].reindex(idx)
# #     else:
# #         if not status:
# #             status = True
# #             idx = datas[i].index.tolist()
# #             for j in redo:
# #                 datas[j].reindex(idx)
# pd.concat(datas, keys=tickers,names=['Ticker', 'Date'])

# # strat = MomentumQuartileStrategy(gen, [slice('2018-10-07','2019-09-07'), slice('2019-09-07','2019-10-05')], weight_recent_returns)
# # strat.save_to_csv("%s" % file_head)   
# # p = Portfolio(initial_capital, strat.dg.tickers)

In [ ]:
class Strategy:
    def __init__(self, data_gen, periods, test_set=None, test_set_df=pd.DataFrame(), returns_data=None):
        self.dg = data_gen
        self.periods = []
        self.backtest = False
        
        if self.dg._calculate:
            for part in periods:
                self.periods.append(self.dg.xs(part))

            if test_set != None:
                self.backtest = True
                self.test_set = self.dg.xs(test_set)
        else: # loading from csv
            self.periods = periods
            self._returns = returns_data
            
            if test_set_df.empty == False:
                self.backtest = True
                self.test_set = test_set_df
    
    def _calc_returns(self):
        pass
    
    def execute(self, portfolio):
        pass
    
    def save_to_csv(self, sid):
        for i in range(len(self.periods)):
            p = self.periods[i]
            p.to_csv("%s_p%d.csv" % (sid, i+1), sep=',')
        self._returns.to_csv("%s_returns.csv" % sid, sep=',')
        
        if self.backtest:
            self.test_set.to_csv("%s_testset.csv" % sid, sep=',')            
        
    @staticmethod
    def _load_from_csv(period_csvs, returns_csv="", test_set=None):
        if returns_csv != "":
            returns_data = pd.read_csv(returns_csv, index_col=[0])
            
        periods = []
        for p in period_csvs:
            periods.append(pd.read_csv(p, index_col3=[0,1]))
            
        if test_set != None:
            test_set = pd.read_csv(test_set, index_col=[0,1])
        else:
            test_set = pd.DataFrame()

        data_gen = DataGenerator(periods[0].index.levels[0].tolist(), periods[0].index.levels[1][0], periods[-1].index.levels[1][-1], calculate=False)        
        return Strategy(data_gen, periods, test_set_df=test_set, returns_data=returns_data)
        
class MomentumQuartileStrategy(Strategy):
    def __init__(self, data_gen, periods, weight_recent_returns, test_set=None):
        super().__init__(data_gen, periods, test_set)
        self.weight_recent_returns = weight_recent_returns
        self._calc_returns()
        
    @staticmethod
    def load_from_csv(period_csvs, weight_recent_returns, returns_csv="", test_set=None):
        strategy = Strategy._load_from_csv(period_csvs, returns_csv, test_set)
        strategy.__class__ = MomentumQuartileStrategy
        strategy.weight_recent_returns = weight_recent_returns
        return strategy
    
    def _calc_returns(self):
        cols = ['Weighted Std Dev', 'Weighted Returns', 'Weighted EAR', 'Weighted Sharpe Ratio', 'Buy Price', 
                                                                                            'Portfolio Weight']
        for i in range(0, len(self.periods)):
            cols.insert(i*3,'Period%d Std Dev' % (i+1))
            cols.insert(i*3 + 1,'Period%d Returns' % (i+1))
            cols.insert(i*3 + 2,'Period%d EAR' % (i+1))
        
        self._returns = pd.DataFrame(0.0, columns=cols, index=self.dg.tickers)

        for t in self.dg.tickers:
            for i in range(len(self.periods)):
                p = self.periods[i]
                p.loc[t].head(1)['Returns'] = 1.0
                self._returns.loc[t]['Period%d Returns' % (i+1)] = (p.loc[t]['Returns']).prod() - 1
                
            self._returns.loc[t]['Buy Price'] = self.periods[-1].loc[t]['Adj Close'].tail(1)

        if len(self.periods) == 1:
            self._returns['Period1 EAR'] = ((self._returns['Period1 Returns'] + 1)**
                                                        (1.0/float(len(self.periods[0].index.levels[1])))) - 1.0
                        
            self._returns["Period1 Std Dev"] = self.periods[0]['Adj Close'].std(level=0)

            self._returns['Weighted Std Dev'] = (1.0 - self.weight_recent_returns) * (self._returns['Period1 Std Dev'])
                                            + self.weight_recent_returns * (self._returns['Period2 Std Dev'])
            self._returns['Weighted Returns'] = self._returns['Period1 Returns']
            self._returns['Weighted EAR'] = self._returns['Period1 EAR']
            
            self._returns['Weighted Sharpe Ratio'] = (self._returns['Weighted Returns'] - risk_free_rate) 
                                                                        / self._returns['Weighted Std Dev']
        elif len(self.periods) == 2:
            self._returns['Period1 EAR'] = ((self._returns['Period1 Returns'] + 1)
                                                    **(1.0/float(len(self.periods[0].index.levels[1])))) - 1.0
            self._returns['Period2 EAR'] = ((self._returns['Period2 Returns'] + 1)
                                                    **(1.0/float(len(self.periods[1].index.levels[1])))) - 1.0

            self._returns["Period1 Std Dev"] = self.periods[0]['Adj Close'].std(level=0)
            self._returns["Period2 Std Dev"] = self.periods[1]['Adj Close'].std(level=0)
            
            self._returns['Weighted Std Dev'] = (1.0 - self.weight_recent_returns) *(self._returns['Period1 Std Dev'])
                                    + self.weight_recent_returns * (self._returns['Period2 Std Dev'])
            self._returns['Weighted Returns'] = (1.0 - self.weight_recent_returns) * (1 + self._returns['Period1 Returns'])
                                    + self.weight_recent_returns * (1 + self._returns['Period2 Returns']) - 1
            self._returns['Weighted EAR'] = (1.0 - self.weight_recent_returns) * (1 + self._returns['Period1 EAR'])
                                    + self.weight_recent_returns * (1 + self._returns['Period2 EAR']) - 1
            
            self._returns['Weighted Sharpe Ratio'] = (self._returns['Weighted Returns'] - risk_free_rate) / self._returns['Weighted Std Dev']
        else:
            print("[ERROR] MomentumQuartileStrategy is only implemented for 1 or 2 periods")
            return
        
        self._returns = self._returns.sort_values(by=['Weighted Returns'])
        
    def execute(self, portfolio):

        q1 = self._returns.iloc[:int(len(self.dg.tickers)/5)] # q1: will take short position (worst performers)
        q5 = self._returns.iloc[int(4*len(self.dg.tickers)/5):] # q5: will take long position (best perfo rmers)
        
        weight = 1.0/(len(q1) + len(q5)) # equal weighting
        
        pf = portfolio.portfolio.copy()
        for t in q1.index.tolist():
            self._returns.loc[t, 'Portfolio Weight'] = weight
            price = portfolio.initial_capital * self._returns.loc[t, 'Portfolio Weight']
            pf.loc[t, 'Invested Positions'] = pf.loc[t, 'Invested Positions'] - price
            pf.loc[t, 'Invested Price'] = self._returns.loc[t, 'Buy Price']
            pf.loc[t, 'Invested Shares'] = pf.loc[t, 'Invested Positions'] / pf.loc[t, 'Invested Price']

            if self.backtest:
                pf.loc[t, 'Final Price'] = self.test_set.loc[t, "Close"].tail(1)[0]
                pf.loc[t, 'Final Positions'] = pf.loc[t, 'Final Price'] * pf.loc[t, 'Invested Shares']
                if pf.loc[t, 'Invested Positions'] < 0: # SHORTING
                    pf.loc[t, 'Final Return'] = 1 - pf.loc[t, 'Final Positions'] / pf.loc[t, 'Invested Positions']
                elif pf.loc[t, 'Invested Positions'] > 0: # LONGING
                    pf.loc[t, 'Final Return'] = pf.loc[t, 'Final Positions'] / pf.loc[t, 'Invested Positions'] - 1
                    
        for t in q5.index.tolist():
            self._returns.loc[t, 'Portfolio Weight'] = weight      
            price = portfolio.initial_capital * self._returns.loc[t, 'Portfolio Weight']
            pf.loc[t, 'Invested Positions'] = pf.loc[t, 'Invested Positions'] + price
            pf.loc[t, 'Invested Price'] = self._returns.loc[t, 'Buy Price']
            pf.loc[t, 'Invested Shares'] = pf.loc[t, 'Invested Positions'] / pf.loc[t, 'Invested Price']

            if self.backtest:
                pf.loc[t, 'Final Price'] = self.test_set.loc[t, "Close"].tail(1)[0]
                pf.loc[t, 'Final Positions'] = pf.loc[t, 'Final Price'] * pf.loc[t, 'Invested Shares']
                if pf.loc[t, 'Invested Positions'] < 0: # SHORTING
                    pf.loc[t, 'Final Return'] = 1 - pf.loc[t, 'Final Positions'] / pf.loc[t, 'Invested Positions']
                elif pf.loc[t, 'Invested Positions'] > 0: # LONGING
                    pf.loc[t, 'Final Return'] = pf.loc[t, 'Final Positions'] / pf.loc[t, 'Invested Positions'] - 1
    
        return pf
        
    def get_returns(self):
        return self._returns
        

In [5]:
class Summary:
    def __init__(self, portfolio):
        self.portfolio = portfolio
        
    def __str__(self):
        inv_pos = int(self.portfolio["Invested Positions"].sum())
        final_pos = int(self.portfolio['Final Positions'].sum())
        ret =  (self.portfolio['Final Return']).sum() / len(self.portfolio)
        ret_h = initial_capital * (ret + 1)
        return ("Portfolio Summary:\n\tInvested Position = %d\n\tFinal Position = %d\n\tReturn = %s %%\n\tHypothetical Profit = %s" % (inv_pos, final_pos, ret*100, ret_h))

class Portfolio:
    def __init__(self, initial_capital, tickers):
        self.initial_capital = initial_capital
        self.cash = initial_capital
        self.portfolio = pd.DataFrame(0.0, columns=['Initial Positions', 'Invested Positions', 'Invested Price', 'Invested Shares', 'Final Price', 'Final Positions', 'Final Return'], index=tickers)
        self.finalized = False
        self.strategies = []
    
    def add_strategies(self, strategies):
        for s in strategies:
            self.strategies.append(s)
        return self
        
    def calculate_portfolio(self):
        
        if len(self.strategies) > 1:
            print("[ERROR] Portfolio->calculate_portfolio() : Can only apply 1 strategy to a portfolio as of now.")
            return
        
        results = []
        for strat in self.strategies:
            results.append(strat.execute(self))
            
        # TODO : aggregate function for strategy execution result array
        self.portfolio = results[0]
        
        self.final_return = (self.portfolio['Final Return'] + 1).prod() - 1
        self.portfolio = self.portfolio.sort_values(by=['Final Return'])
        self.finalized = True
        return self
    
    def summary(self):
        if not self.finalized:
            print("[WARN] Portfolio.summary() -> attempting to calculate summary on non-finalized portfolio.")
            return None
        return Summary(self)

## Utility Functions

In [6]:
## SHORTING IMPLIED VOLATILITY (at home trading) --> now long volatility! <- research!
def sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)

    tickers = [e.replace('.','-') for e in tickers]

    return [e[:len(e)-1] for e in tickers]

## [Script] Get stock data for S&P 500

In [7]:
# Global Variables
initial_capital = 100000.0
weight_recent_returns = .2
file_head = "mqs_10-5-19_test"

# User Methods
def gen_predictive_mqs():
    tickers = sp500_tickers()
    tickers.remove('BKR')
    gen = DataGenerator(tickers, datetime.datetime(2018, 10, 7), datetime.datetime(2019, 10, 5))
    strat = MomentumQuartileStrategy(gen, [slice('2018-10-07','2019-09-07'), slice('2019-09-07','2019-10-05')], weight_recent_returns)
    strat.save_to_csv("%s" % file_head)   
    return Portfolio(initial_capital, strat.dg.tickers)

def gen_backtest_mqs():
    tickers = sp500_tickers()
    tickers.remove('BKR')
    gen = DataGenerator(tickers, datetime.datetime(2018, 10, 7), datetime.datetime(2019, 10, 5))
    strat = MomentumQuartileStrategy(gen, [slice('2018-10-07','2019-08-10'), slice('2019-08-10','2019-09-07')], weight_recent_returns, test_set=slice('2019-09-07','2019-10-05'))
    strat.save_to_csv("%s_backtest" % file_head)
    return Portfolio(initial_capital, strat.dg.tickers)

def load_predictive_mqs():
    strat = MomentumQuartileStrategy.load_from_csv(["%s_p1.csv" % file_head, "%s_p2.csv" % file_head], weight_recent_returns, "%s_returns.csv" % file_head)
    return Portfolio(initial_capital, strat.dg.tickers).add_strategies([strat]).calculate_portfolio()
    
def load_backtest_mqs():
    strat = MomentumQuartileStrategy.load_from_csv(["%s_p1.csv" % file_head, "%s_p2.csv" % file_head], weight_recent_returns, "%s_returns.csv" % file_head, test_set="%s_testset.csv" % file_head)
    return Portfolio(initial_capital, strat.dg.tickers).add_strategies([strat]).calculate_portfolio()


In [8]:
# adjust for splits and dividends --> 
# fist look at distr, especially if extra big values
# adjust for winsorization and splits!!!! extremes! --> replace w edge 

## how are u averaging returns **** key TODO

In [9]:
# MODEL 1 --> BACKTEST 1
# file_head = "mqs_10-4-19_test"
# strat = MomentumQuartileStrategy.load_from_csv(["%s_p1.csv" % file_head, "%s_p2.csv" % file_head], weight_recent_returns, "%s_returns.csv" % file_head)#, test_set="%s_testset.csv" % file_head)
# p = Portfolio(initial_capital, strat.dg.tickers).add_strategies([strat]).calculate_portfolio()
# p.portfolio['Invested Positions']

# MODEL 2 --> PREDICTION 1
# pred = load_predictive_mqs()
# file_head = "mqs_10-4-19_test"
# bt = load_backtest_mqs()

In [10]:
# MODEL 3 -->
# file_head = "model3_backtest"
# bt = load_backtest_mqs()
file_head = "model3"
pred = gen_predictive_mqs()
pred.portfolio
# print(Summary(bt.portfolio))

KeyboardInterrupt: 

In [ ]:
#pred.portfolio.to_csv("MODEL.csv")
# pred.portfolio

In [ ]:
class RealPortfolio:
    def __init__(self, tickers, data_gen):
        self.tickers = tickers
        self.gen = data_gen
        self.holdings = self.gen.data.drop(['High', 'Low'], index=)


In [ ]:
stocks = ["AAPL","AMZN","BAL","BRFS","BURL","CHIE","CME","COST","CRON","CRWD","CTZ","EA","FISV","FUJHY","HEES","HPE","KL","LN","MSCI","MSFT","MSTR","MU","NTT","ONEY","PSTG","RSG","SHAK","SNAP","SOYB","SPOT","SQ","SSO","TTWO","UAA","V","VBR","XMLV","ZTS"]
gen = DataGenerator(stocks, datetime.datetime(2019, 9, 23), datetime.datetime(2019, 11, 8))

In [ ]:
gen.data.to_csv('output.csv')